In [29]:
import requests
import os
from tqdm import tqdm

def get_wikipedia_page(title, get_links=False):
    """
    Fetches the content and links of a Wikipedia page given its title.

    Args:
        title (str): The title of the Wikipedia page.

    Returns:
        dict: A dictionary containing the page content and links.
    """
    S = requests.Session()

    URL = "https://en.wikipedia.org/w/api.php"

    # First, get the content of the page
    PARAMS_CONTENT = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "titles": title,
        "explaintext": True,
    }

    response_content = S.get(url=URL, params=PARAMS_CONTENT)
    data_content = response_content.json()

    pages_content = data_content['query']['pages']
    page = next(iter(pages_content.values()))
    page_content = page.get('extract', '')

    # Next, get the links from the page
    PARAMS_LINKS = {
        "action": "query",
        "format": "json",
        "prop": "links",
        "titles": title,
        "pllimit": "max",
    }

    if get_links:
        links = []
        while True:
            response_links = S.get(url=URL, params=PARAMS_LINKS)
            data_links = response_links.json()
            pages_links = data_links['query']['pages']
            page_links = next(iter(pages_links.values()))
            if 'links' in page_links:
                links.extend([link['title'] for link in page_links['links']])

            # Check if there is a continuation
            if 'continue' in data_links:
                PARAMS_LINKS.update(data_links['continue'])
            else:
                break

        return {
            'title': page.get('title'),
            'content': page_content,
            'links': links
        }
    else:
        return {
            'title': page.get('title'),
            'content': page_content,
        }

def get_pages_content(titles):
    """
    Fetches the content of multiple Wikipedia pages.

    Args:
        titles (list): A list of Wikipedia page titles.

    Returns:
        dict: A dictionary mapping page titles to their content.
    """
    S = requests.Session()
    URL = "https://en.wikipedia.org/w/api.php"

    contents = {}
    for title in titles:
        PARAMS = {
            "action": "query",
            "format": "json",
            "prop": "extracts",
            "titles": title,
            "explaintext": True,
        }
        response = S.get(url=URL, params=PARAMS)
        data = response.json()
        pages = data['query']['pages']
        page = next(iter(pages.values()))
        content = page.get('extract', '')
        contents[title] = content

    return contents

def save_content_to_file(title, content, directory='output/History of Pittsburgh/'):
    """
    Saves the content to a text file named after the page title.

    Args:
        title (str): The title of the Wikipedia page.
        content (str): The content to be saved.
        directory (str): The directory where the file will be saved.
    """
    # Ensure the output directory exists
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Create a safe file name by replacing or removing invalid characters
    safe_title = "".join(c for c in title if c.isalnum() or c in (" ", "_")).rstrip()
    file_name = f"{safe_title}.txt"
    file_path = os.path.join(directory, file_name)

    # Write content to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

    # print(f"Content saved to {file_path}")




if __name__ == "__main__":
    page_title = "Railway air brake"
    main_page = get_wikipedia_page(page_title, get_links=True)

    # Output the main page content
    save_content_to_file(main_page['title'], main_page['content'])

    # Now, get the content of the subtopics (links)
    subtopics = list(set(main_page['links']))
    for topic in tqdm(subtopics):
        print(f"Fetching content for {topic}...")
        content = get_wikipedia_page(topic)
        save_content_to_file(content['title'], content['content'])


In [30]:
import requests
from bs4 import BeautifulSoup
import os
import urllib.parse
from tqdm import tqdm

def get_wikipedia_page_content(title):
    """
    Fetches the plain text content of a Wikipedia page given its title.

    Args:
        title (str): The title of the Wikipedia page.

    Returns:
        str: The plain text content of the page.
    """
    S = requests.Session()
    URL = "https://en.wikipedia.org/w/api.php"

    PARAMS = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "titles": title,
        "explaintext": True,
    }

    response = S.get(url=URL, params=PARAMS)
    data = response.json()

    pages = data['query']['pages']
    page = next(iter(pages.values()))
    page_content = page.get('extract', '')

    return page_content

def get_wikipedia_page_links(title):
    """
    Retrieves all unique Wikipedia page titles linked from the specified page,
    using the actual URLs to ensure accurate titles.

    Args:
        title (str): The title of the Wikipedia page.

    Returns:
        list: A list of linked page titles.
    """
    S = requests.Session()
    URL = f"https://en.wikipedia.org/wiki/{urllib.parse.quote(title)}"

    response = S.get(URL)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract all anchor tags within the main content area
    content_div = soup.find('div', {'id': 'mw-content-text'})
    links = content_div.find_all('a', href=True)

    page_titles = set()
    for link in links:
        href = link['href']
        if href.startswith('/wiki/') and not href.startswith('/wiki/Special:'):
            # Extract the title from the URL
            linked_title = href.split('/wiki/')[-1]
            # Decode URL-encoded characters
            linked_title = urllib.parse.unquote(linked_title)
            # Skip links to sections within the same page
            if '#' in linked_title:
                linked_title = linked_title.split('#')[0]
            # Skip empty titles
            if linked_title:
                page_titles.add(linked_title)

    return list(page_titles)

def save_content_to_file(title, content, directory='output/History of Pittsburgh/'):
    """
    Saves the content to a text file named after the page title.

    Args:
        title (str): The title of the Wikipedia page.
        content (str): The content to be saved.
        directory (str): The directory where the file will be saved.
    """
    # Ensure the output directory exists
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Create a safe file name by replacing or removing invalid characters
    safe_title = "".join(c for c in title if c.isalnum() or c in (" ", "_", "-", "'")).rstrip()
    file_name = f"{safe_title}.txt"
    file_path = os.path.join(directory, file_name)

    # Write content to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

    # print(f"Content saved to {file_path}")

if __name__ == "__main__":
    main_page_title = "History of Pittsburgh"
    print(f"Fetching content for: {main_page_title}")

    # Step 1: Get and save the main page content
    main_content = get_wikipedia_page_content(main_page_title)
    save_content_to_file(main_page_title, main_content)

    # Step 2: Get linked page titles using actual URLs
    linked_titles = get_wikipedia_page_links(main_page_title)
    print(f"Found {len(linked_titles)} linked pages.")

    # Optional: Limit the number of linked pages to process
    # linked_titles = linked_titles[:10]

    # Step 3: Fetch and save content for each linked page
    for title in tqdm(linked_titles):
        # print(f"Fetching content for: {title}")
        content = get_wikipedia_page_content(title)
        save_content_to_file(title, content)


Fetching content for: History of Pittsburgh
Content saved to output/History of Pittsburgh/History of Pittsburgh.txt
Found 408 linked pages.


  0%|          | 1/408 [00:00<01:12,  5.60it/s]

Content saved to output/History of Pittsburgh/Archaeological_site.txt


  0%|          | 2/408 [00:00<01:42,  3.95it/s]

Content saved to output/History of Pittsburgh/Smallpox.txt


...

Content saved to output/History of Pittsburgh/List_of_tallest_educational_buildings.txt
Content saved to output/History of Pittsburgh/List_of_nicknames_for_Pittsburgh.txt


100%|█████████▉| 406/408 [01:29<00:00,  4.54it/s]

Content saved to output/History of Pittsburgh/Gulf_Oil.txt
Content saved to output/History of Pittsburgh/Paleo-Indian.txt


100%|██████████| 408/408 [01:29<00:00,  4.56it/s]

Content saved to output/History of Pittsburgh/Work-rule.txt
Content saved to output/History of Pittsburgh/Coal_dust.txt


# Pittsburgh Events

##### Scrape all texts, urls and events without ``show more''

In [4]:
import requests
from bs4 import BeautifulSoup
import os
import urllib.parse
from tqdm import tqdm
import csv

def extract_events(soup, output_path):
    """
    Extracts event details (date, time, location) from the parsed HTML soup
    and saves them into 'events.csv' in the output_path.
    
    Args:
        soup (BeautifulSoup): The parsed HTML content.
        output_path (str): The directory where output files will be saved.
    """
    events = []
    
    # Find the unordered list with class 'dates-list'
    dates_list = soup.find('ul', class_='dates-list')
    
    if not dates_list:
        print("No events found in the HTML content.")
        return
    
    # Loop through each list item with class 'date-row'
    for date_row in dates_list.find_all('li', class_='date-row'):
        # Extract date information
        date_div = date_row.find('div', class_='date')
        if date_div:
            month = date_div.find('div', class_='month').get_text(strip=True)
            day = date_div.find('div', class_='day').get_text(strip=True)
            year = date_div.find('div', class_='year').get_text(strip=True)
            event_date = f"{month} {day}, {year}"
        else:
            event_date = 'N/A'
        
        # Extract time information
        time_div = date_row.find('div', class_='time')
        if time_div:
            time_text = time_div.contents[0].strip()
        else:
            time_text = 'N/A'
        
        # Extract event name and venue
        venue_div = date_row.find('div', class_='venue')
        if venue_div:
            # Event Name
            event_name_div = venue_div.find('div')
            event_name = event_name_div.get_text(strip=True) if event_name_div else 'N/A'
            
            # Venue Name
            venue_name_div = venue_div.find('div', class_='date-desc')
            venue_name = venue_name_div.get_text(strip=True) if venue_name_div else 'N/A'
            
            # Location
            location_span = venue_div.find('span', class_='location')
            location = location_span.get_text(strip=True) if location_span else 'N/A'
        else:
            event_name = 'N/A'
            venue_name = 'N/A'
            location = 'N/A'
        
        # Extract price
        price_div = date_row.find('div', class_='from-price')
        price = price_div.get_text(strip=True) if price_div else 'N/A'
        
        # Extract ticket link
        ticket_link_tag = date_row.find('a', text='Tickets')
        ticket_link = ticket_link_tag['href'] if ticket_link_tag else 'N/A'
        
        # Store the extracted data
        event = {
            'Date': event_date,
            'Time': time_text,
            'Event Name': event_name,
            'Venue': venue_name,
            'Location': location,
            'Price': price,
            'Ticket Link': ticket_link
        }
        events.append(event)
    
    # Save events to a CSV file
    if events:
        events_file = os.path.join(output_path, 'events.csv')
        keys = ['Date', 'Time', 'Event Name', 'Venue', 'Location', 'Price', 'Ticket Link']
        with open(events_file, 'w', newline='', encoding='utf-8') as f:
            dict_writer = csv.DictWriter(f, fieldnames=keys)
            dict_writer.writeheader()
            dict_writer.writerows(events)
        print(f"Extracted {len(events)} events and saved to {events_file}")
    else:
        print("No events were extracted.")


def scrape_website(url, output_path):
    """
    Given a URL and an output path, this function:
    - Finds all paragraphs on the webpage and stores them into 'paragraphs.txt'.
    - Finds all links on the webpage and stores them into 'url.txt'.
    - Downloads all PDFs linked on the webpage into the 'downloaded_files' folder.
    
    Args:
        url (str): The URL of the webpage to scrape.
        output_path (str): The directory where output files and folders will be saved.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract all paragraphs and save to paragraphs.txt
        paragraphs = soup.find_all('p')
        paragraphs_text = [para.get_text(strip=True) for para in paragraphs if para.get_text(strip=True)]
        paragraphs_file = os.path.join(output_path, 'paragraphs.txt')
        with open(paragraphs_file, 'w', encoding='utf-8') as f:
            for para in paragraphs_text:
                f.write(para + '\n\n')
        print(f"Paragraphs saved to {paragraphs_file}")
        
        # Extract all links and save to url.txt
        links = []
        for link in soup.find_all('a', href=True):
            href = link['href']
            full_url = urllib.parse.urljoin(url, href)
            links.append(full_url)
        links_file = os.path.join(output_path, 'url.txt')
        with open(links_file, 'w', encoding='utf-8') as f:
            for link in links:
                f.write(link + '\n')
        # print(f"Links saved to {links_file}")
        
        # Download files linked on the webpage
        file_extensions = ('.pdf', '.docx')
        file_links = [link for link in links if link.lower().endswith(file_extensions)]
        if (len(file_links) > 0):
            downloads_folder = os.path.join(output_path, 'downloaded_files')
            if not os.path.exists(downloads_folder):
                os.makedirs(downloads_folder)
        for file_link in file_links:
            file_url = file_link
            file_name = os.path.basename(urllib.parse.urlsplit(file_url).path)
            file_path = os.path.join(downloads_folder, file_name)
            # print(f"Downloading {file_name}...")
            file_response = requests.get(file_url)
            file_response.raise_for_status()
            with open(file_path, 'wb') as f:
                f.write(file_response.content)
        

        # Extract event details and save to events.csv
        extract_events(soup, output_path)


    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")


# URL of the webpage to scrape
url = 'https://www.britannica.com/place/Pittsburgh'
url = 'https://pittsburghpa.gov/events/'
url = 'https://www.visitpittsburgh.com/'
url = 'https://www.cmu.edu/about/'
url = 'https://pittsburgh.events/october/'
url = 'https://pittsburgh.events/november/'
url = 'https://pittsburgh.events/december/'
url = 'https://pittsburgh.events/january/'
url = 'https://pittsburgh.events/february/'
url = 'https://pittsburgh.events/march/'
url = 'https://pittsburgh.events/april/'
url = 'https://pittsburgh.events/may/'
url = 'https://pittsburgh.events/june/'
url = 'https://pittsburgh.events/july/'
url = 'https://pittsburgh.events/august/'
url = 'https://pittsburgh.events/september/'

scrape_website(url, './output/pittsburgh_events/')

# Read urls in a text file and scrape each one
with open('./output/downtown_events/url.txt', 'r') as file:
    urls = file.readlines()
    for url in tqdm(urls):
        url = url.strip()
        title = url.split('//')[-1].replace('/', '_')
        scrape_website(url, f'./output/downtown_events/children/{title}')



Paragraphs saved to ./output/pittsburgh_events_oct/paragraphs.txt
Extracted 30 events and saved to ./output/pittsburgh_events_oct/events.csv


/var/folders/jt/kpshpf8d1991jhtxl__hj8p00000gn/T/ipykernel_9164/2469560081.py:69: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  ticket_link_tag = date_row.find('a', text='Tickets')


#### Scrape all events

In [15]:
import os
import urllib.parse
import csv
import time
import random
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
)
from bs4 import BeautifulSoup

def extract_events(soup, output_path, url):
    """
    Extracts event details (date, time, location) from the parsed HTML soup
    and saves them into 'events.csv' in the output_path.

    Args:
        soup (BeautifulSoup): The parsed HTML content.
        output_path (str): The directory where output files will be saved.
        url (str): The base URL for resolving relative links.
    """
    events = []

    # Find the unordered list with class 'dates-list'
    dates_list = soup.find('ul', class_='dates-list')

    if not dates_list:
        print("No events found in the HTML content.")
        return

    # Loop through each list item with class 'date-row'
    for date_row in dates_list.find_all('li', class_='date-row'):
        # Extract date information
        date_div = date_row.find('div', class_='date')
        if date_div:
            month = date_div.find('div', class_='month').get_text(strip=True)
            day = date_div.find('div', class_='day').get_text(strip=True)
            year = date_div.find('div', class_='year').get_text(strip=True)
            event_date = f"{month} {day}, {year}"
        else:
            event_date = 'N/A'

        # Extract time information
        time_div = date_row.find('div', class_='time')
        if time_div:
            # Assuming time is the first text node
            time_text = time_div.contents[0].strip()
        else:
            time_text = 'N/A'

        # Extract event name and venue
        venue_div = date_row.find('div', class_='venue')
        if venue_div:
            # Event Name
            event_name_div = venue_div.find('div')
            event_name = event_name_div.get_text(strip=True) if event_name_div else 'N/A'

            # Venue Name
            venue_name_div = venue_div.find('div', class_='date-desc')
            venue_name = venue_name_div.get_text(strip=True) if venue_name_div else 'N/A'

            # Location
            location_span = venue_div.find('span', class_='location')
            location = location_span.get_text(strip=True) if location_span else 'N/A'
        else:
            event_name = 'N/A'
            venue_name = 'N/A'
            location = 'N/A'

        # Extract price
        price_div = date_row.find('div', class_='from-price')
        price = price_div.get_text(strip=True) if price_div else 'N/A'

        # Extract ticket link
        ticket_link_tag = date_row.find('a', text='Tickets')
        ticket_link = ticket_link_tag['href'] if ticket_link_tag else 'N/A'
        if ticket_link != 'N/A':
            ticket_link = urllib.parse.urljoin(url, ticket_link)

        # Store the extracted data
        event = {
            'Date': event_date,
            'Time': time_text,
            'Event Name': event_name,
            'Venue': venue_name,
            'Location': location,
            'Price': price,
            'Ticket Link': ticket_link
        }
        events.append(event)

    # Save events to a CSV file
    if events:
        events_file = os.path.join(output_path, 'events.csv')
        keys = ['Date', 'Time', 'Event Name', 'Venue', 'Location', 'Price', 'Ticket Link']
        with open(events_file, 'w', newline='', encoding='utf-8') as f:
            dict_writer = csv.DictWriter(f, fieldnames=keys)
            dict_writer.writeheader()
            dict_writer.writerows(events)
        print(f"Extracted {len(events)} events and saved to {events_file}")
    else:
        print("No events were extracted.")

def scrape_website(url, output_path):
    """
    Given a URL and an output path, this function:
    - Loads the webpage using Selenium and clicks 'Show More' until all events are loaded.
    - Extracts event details and saves them into 'events.csv'.

    Args:
        url (str): The URL of the webpage to scrape.
        output_path (str): The directory where output files and folders will be saved.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Set up Selenium WebDriver using webdriver-manager
    options = Options()
    options.headless = False  # Set to True after debugging
    # Optional: Set user-agent to mimic a real browser
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/112.0.0.0 Safari/537.36"
    )

    try:
        # Initialize WebDriver with webdriver-manager
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        print("WebDriver initialized successfully.")
    except Exception as e:
        print(f"Error initializing WebDriver: {e}")
        return

    try:
        print(f"Navigating to {url}")
        driver.get(url)
        wait = WebDriverWait(driver, 10)

        while True:
            try:
                print("Looking for the 'Show More' button...")
                # XPath selector to find the span with exact text "Show More"
                show_more_xpath = '//span[text()="Show More"]/parent::*'

                # Wait until the 'Show More' button is present and clickable
                show_more_button = wait.until(
                    EC.element_to_be_clickable((By.XPATH, show_more_xpath))
                )
                if show_more_button:
                    print("'Show More' button found. Scrolling into view...")
                    driver.execute_script("arguments[0].scrollIntoView({ behavior: 'smooth', block: 'center' });", show_more_button)
                    time.sleep(random.uniform(1, 2))  # Random sleep to mimic human behavior

                    print("Clicking the 'Show More' button...")
                    show_more_button.click()
                    print("Clicked 'Show More'. Waiting for new content to load...")
                    time.sleep(random.uniform(3, 6))  # Random sleep to allow content to load
            except (NoSuchElementException, TimeoutException):
                print("No more 'Show More' buttons found or an error occurred.")
                break
            except ElementClickInterceptedException:
                print("Click intercepted. Retrying after a short delay...")
                time.sleep(2)
                continue
            except StaleElementReferenceException:
                print("Stale element reference. Retrying after a short delay...")
                time.sleep(2)
                continue
            except Exception as e:
                print(f"An unexpected error occurred while clicking 'Show More': {e}")
                break

        # Get the page source after all events are loaded
        print("Retrieving page source...")
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # Extract event details and save to events.csv
        print("Extracting event details...")
        extract_events(soup, output_path, url)

    except Exception as e:
        print(f"An error occurred during scraping: {e}")

    finally:
        print("Closing the WebDriver.")
        driver.quit()

if __name__ == "__main__":
    # URL of the webpage to scrape
    url = 'https://pittsburgh.events/'  # Replace with your actual URL
    output_path = './output/pittsburgh_events/'
    scrape_website(url, output_path)


WebDriver initialized successfully.
Navigating to https://pittsburgh.events/
Looking for the 'Show More' button...
'Show More' button found. Scrolling into view...
Clicking the 'Show More' button...
Clicked 'Show More'. Waiting for new content to load...
Looking for the 'Show More' button...
'Show More' button found. Scrolling into view...
Clicking the 'Show More' button...
Click intercepted. Retrying after a short delay...
Looking for the 'Show More' button...
'Show More' button found. Scrolling into view...
Clicking the 'Show More' button...
Clicked 'Show More'. Waiting for new content to load...
Looking for the 'Show More' button...
'Show More' button found. Scrolling into view...
Clicking the 'Show More' button...
Clicked 'Show More'. Waiting for new content to load...
Looking for the 'Show More' button...
'Show More' button found. Scrolling into view...
Clicking the 'Show More' button...
Clicked 'Show More'. Waiting for new content to load...
Looking for the 'Show More' button...


/var/folders/jt/kpshpf8d1991jhtxl__hj8p00000gn/T/ipykernel_12451/190630264.py:85: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  ticket_link_tag = date_row.find('a', text='Tickets')


#### Scrape all events, texts, urls and file 

In [16]:
import os
import urllib.parse
import csv
import time
import random
import logging
import requests
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
)
from bs4 import BeautifulSoup

# Configure logging
logging.basicConfig(
    filename='scraping.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def extract_events(soup, output_path, url):
    """
    Extracts event details (date, time, location) from the parsed HTML soup
    and saves them into 'events.csv' in the output_path.

    Args:
        soup (BeautifulSoup): The parsed HTML content.
        output_path (str): The directory where output files will be saved.
        url (str): The base URL for resolving relative links.
    """
    events = []

    # Find the unordered list with class 'dates-list'
    dates_list = soup.find('ul', class_='dates-list')

    if not dates_list:
        logging.warning("No events found in the HTML content.")
        print("No events found in the HTML content.")
        return

    # Loop through each list item with class 'date-row'
    for date_row in dates_list.find_all('li', class_='date-row'):
        # Extract date information
        date_div = date_row.find('div', class_='date')
        if date_div:
            month = date_div.find('div', class_='month').get_text(strip=True)
            day = date_div.find('div', class_='day').get_text(strip=True)
            year = date_div.find('div', class_='year').get_text(strip=True)
            event_date = f"{month} {day}, {year}"
        else:
            event_date = 'N/A'

        # Extract time information
        time_div = date_row.find('div', class_='time')
        if time_div:
            # Assuming time is the first text node
            time_text = time_div.contents[0].strip()
        else:
            time_text = 'N/A'

        # Extract event name and venue
        venue_div = date_row.find('div', class_='venue')
        if venue_div:
            # Event Name
            event_name_div = venue_div.find('div')
            event_name = event_name_div.get_text(strip=True) if event_name_div else 'N/A'

            # Venue Name
            venue_name_div = venue_div.find('div', class_='date-desc')
            venue_name = venue_name_div.get_text(strip=True) if venue_name_div else 'N/A'

            # Location
            location_span = venue_div.find('span', class_='location')
            location = location_span.get_text(strip=True) if location_span else 'N/A'
        else:
            event_name = 'N/A'
            venue_name = 'N/A'
            location = 'N/A'

        # Extract price
        price_div = date_row.find('div', class_='from-price')
        price = price_div.get_text(strip=True) if price_div else 'N/A'

        # Extract ticket link
        ticket_link_tag = date_row.find('a', text='Tickets')
        ticket_link = ticket_link_tag['href'] if ticket_link_tag else 'N/A'
        if ticket_link != 'N/A':
            ticket_link = urllib.parse.urljoin(url, ticket_link)

        # Store the extracted data
        event = {
            'Date': event_date,
            'Time': time_text,
            'Event Name': event_name,
            'Venue': venue_name,
            'Location': location,
            'Price': price,
            'Ticket Link': ticket_link
        }
        events.append(event)

    # Save events to a CSV file
    if events:
        events_file = os.path.join(output_path, 'events.csv')
        keys = ['Date', 'Time', 'Event Name', 'Venue', 'Location', 'Price', 'Ticket Link']
        with open(events_file, 'w', newline='', encoding='utf-8') as f:
            dict_writer = csv.DictWriter(f, fieldnames=keys)
            dict_writer.writeheader()
            dict_writer.writerows(events)
        logging.info(f"Extracted {len(events)} events and saved to {events_file}")
        print(f"Extracted {len(events)} events and saved to {events_file}")
    else:
        logging.warning("No events were extracted.")
        print("No events were extracted.")

def scrape_links(soup, output_path):
    """
    Extracts all hyperlinks from the parsed HTML soup and saves them to 'url.txt'.

    Args:
        soup (BeautifulSoup): The parsed HTML content.
        output_path (str): The directory where output files will be saved.
    """
    links = set()
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        # Resolve relative URLs
        full_url = urllib.parse.urljoin('https://pittsburgh.events/', href)
        links.add(full_url)

    # Save links to url.txt
    url_file = os.path.join(output_path, 'url.txt')
    with open(url_file, 'w', encoding='utf-8') as f:
        for link in sorted(links):
            f.write(f"{link}\n")
    logging.info(f"Extracted {len(links)} links and saved to {url_file}")
    print(f"Extracted {len(links)} links and saved to {url_file}")

def download_files(soup, output_path):
    """
    Downloads PDF and DOC/DOCX files linked in the parsed HTML soup.

    Args:
        soup (BeautifulSoup): The parsed HTML content.
        output_path (str): The directory where output files will be saved.
    """
    # Create downloaded_files directory
    download_dir = os.path.join(output_path, 'downloaded_files')
    os.makedirs(download_dir, exist_ok=True)

    # Supported file extensions
    supported_extensions = ['.pdf', '.doc', '.docx']

    # Find all links with supported extensions
    files = set()
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        parsed_href = urllib.parse.urlparse(href)
        _, ext = os.path.splitext(parsed_href.path)
        if ext.lower() in supported_extensions:
            full_url = urllib.parse.urljoin('https://pittsburgh.events/', href)
            files.add(full_url)

    logging.info(f"Found {len(files)} files to download.")
    print(f"Found {len(files)} files to download.")

    for file_url in tqdm(files, desc="Downloading files"):
        try:
            response = requests.get(file_url, stream=True, timeout=30)
            response.raise_for_status()

            # Extract the file name
            filename = os.path.basename(urllib.parse.urlparse(file_url).path)
            file_path = os.path.join(download_dir, filename)

            # Avoid downloading the same file multiple times
            if os.path.exists(file_path):
                logging.info(f"File already exists: {file_path}. Skipping download.")
                continue

            # Write the file in chunks
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            logging.info(f"Downloaded file: {file_path}")
        except requests.exceptions.RequestException as e:
            logging.error(f"Failed to download {file_url}: {e}")
            print(f"Failed to download {file_url}: {e}")

def extract_paragraphs(soup, output_path):
    """
    Extracts all paragraph texts from the parsed HTML soup and saves them to 'paragraphs.txt'.

    Args:
        soup (BeautifulSoup): The parsed HTML content.
        output_path (str): The directory where output files will be saved.
    """
    paragraphs = []
    for p_tag in soup.find_all(['p', 'div', 'span'], text=True):
        text = p_tag.get_text(separator=' ', strip=True)
        if text:
            paragraphs.append(text)

    # Save paragraphs to paragraphs.txt
    paragraphs_file = os.path.join(output_path, 'paragraphs.txt')
    with open(paragraphs_file, 'w', encoding='utf-8') as f:
        for para in paragraphs:
            f.write(f"{para}\n")
    logging.info(f"Extracted {len(paragraphs)} paragraphs and saved to {paragraphs_file}")
    print(f"Extracted {len(paragraphs)} paragraphs and saved to {paragraphs_file}")

def scrape_website(url, output_path):
    """
    Given a URL and an output path, this function:
    - Loads the webpage using Selenium and clicks 'Show More' until all events are loaded.
    - Extracts event details and saves them into 'events.csv'.
    - Extracts all links and saves them into 'url.txt'.
    - Downloads PDF and DOC/DOCX files into 'downloaded_files'.
    - Extracts all paragraph texts and saves them into 'paragraphs.txt'.

    Args:
        url (str): The URL of the webpage to scrape.
        output_path (str): The directory where output files and folders will be saved.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        logging.info(f"Created output directory at {output_path}")
        print(f"Created output directory at {output_path}")

    # Set up Selenium WebDriver using webdriver-manager
    options = Options()
    options.headless = False  # Set to True for headless mode
    # Optional: Set user-agent to mimic a real browser
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/112.0.0.0 Safari/537.36"
    )

    try:
        # Initialize WebDriver with webdriver-manager
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        logging.info("WebDriver initialized successfully.")
        print("WebDriver initialized successfully.")
    except Exception as e:
        logging.error(f"Error initializing WebDriver: {e}")
        print(f"Error initializing WebDriver: {e}")
        return

    try:
        logging.info(f"Navigating to {url}")
        print(f"Navigating to {url}")
        driver.get(url)
        wait = WebDriverWait(driver, 10)

        while True:
            try:
                logging.info("Looking for the 'Show More' button...")
                print("Looking for the 'Show More' button...")

                # XPath selector to find the span with exact text "Show More" and its parent
                show_more_xpath = '//span[text()="Show More"]/parent::*'

                # Wait until the 'Show More' button is present and clickable
                show_more_button = wait.until(
                    EC.element_to_be_clickable((By.XPATH, show_more_xpath))
                )

                if show_more_button:
                    logging.info("'Show More' button found. Scrolling into view...")
                    print("'Show More' button found. Scrolling into view...")
                    driver.execute_script("arguments[0].scrollIntoView({ behavior: 'smooth', block: 'center' });", show_more_button)
                    time.sleep(random.uniform(1, 2))  # Random sleep to mimic human behavior

                    logging.info("Clicking the 'Show More' button...")
                    print("Clicking the 'Show More' button...")
                    show_more_button.click()
                    logging.info("Clicked 'Show More'. Waiting for new content to load...")
                    print("Clicked 'Show More'. Waiting for new content to load...")
                    time.sleep(random.uniform(3, 6))  # Random sleep to allow content to load
            except (NoSuchElementException, TimeoutException):
                logging.info("No more 'Show More' buttons found or an error occurred.")
                print("No more 'Show More' buttons found or an error occurred.")
                break
            except ElementClickInterceptedException:
                logging.warning("Click intercepted. Retrying after a short delay...")
                print("Click intercepted. Retrying after a short delay...")
                time.sleep(2)
                continue
            except StaleElementReferenceException:
                logging.warning("Stale element reference. Retrying after a short delay...")
                print("Stale element reference. Retrying after a short delay...")
                time.sleep(2)
                continue
            except Exception as e:
                logging.error(f"An unexpected error occurred while clicking 'Show More': {e}")
                print(f"An unexpected error occurred while clicking 'Show More': {e}")
                break

        # Get the page source after all events are loaded
        logging.info("Retrieving page source...")
        print("Retrieving page source...")
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # Extract event details and save to events.csv
        logging.info("Extracting event details...")
        print("Extracting event details...")
        extract_events(soup, output_path, url)

        # Extract all links and save to url.txt
        logging.info("Extracting all links...")
        print("Extracting all links...")
        scrape_links(soup, output_path)

        # Download PDF and DOC/DOCX files
        logging.info("Downloading PDF and DOC/DOCX files...")
        print("Downloading PDF and DOC/DOCX files...")
        download_files(soup, output_path)

        # Extract all paragraphs and save to paragraphs.txt
        logging.info("Extracting all paragraphs...")
        print("Extracting all paragraphs...")
        extract_paragraphs(soup, output_path)

    except Exception as e:
        logging.error(f"An error occurred during scraping: {e}")
        print(f"An error occurred during scraping: {e}")

    finally:
        logging.info("Closing the WebDriver.")
        print("Closing the WebDriver.")
        driver.quit()

if __name__ == "__main__":
    # URL of the webpage to scrape
    url = 'https://pittsburgh.events/'  # Replace with your actual URL
    output_path = './output/pittsburgh_events/'
    scrape_website(url, output_path)

    # Read URLs from 'url.txt' and scrape each one
    url_txt_path = os.path.join(output_path, 'url.txt')
    if os.path.exists(url_txt_path):
        logging.info(f"Found URL file at {url_txt_path}. Starting to scrape child URLs.")
        print(f"Found URL file at {url_txt_path}. Starting to scrape child URLs.")
        with open(url_txt_path, 'r') as file:
            urls = file.readlines()
            for url in tqdm(urls, desc="Scraping child URLs"):
                url = url.strip()
                if not url:
                    continue
                title = url.split('//')[-1].replace('/', '_')
                child_output_path = f'./output/downtown_events/children/{title}'
                scrape_website(url, child_output_path)
    else:
        logging.warning(f"URL file not found at {url_txt_path}")
        print(f"URL file not found at {url_txt_path}")


Created output directory at ./output/pittsburgh_events/
WebDriver initialized successfully.
Navigating to https://pittsburgh.events/
Looking for the 'Show More' button...
'Show More' button found. Scrolling into view...
Clicking the 'Show More' button...
Clicked 'Show More'. Waiting for new content to load...
Looking for the 'Show More' button...
'Show More' button found. Scrolling into view...
Clicking the 'Show More' button...
Clicked 'Show More'. Waiting for new content to load...
Looking for the 'Show More' button...
'Show More' button found. Scrolling into view...
Clicking the 'Show More' button...
Clicked 'Show More'. Waiting for new content to load...
Looking for the 'Show More' button...
'Show More' button found. Scrolling into view...
Clicking the 'Show More' button...
Clicked 'Show More'. Waiting for new content to load...
Looking for the 'Show More' button...
'Show More' button found. Scrolling into view...
Clicking the 'Show More' button...
Clicked 'Show More'. Waiting for 

/var/folders/jt/kpshpf8d1991jhtxl__hj8p00000gn/T/ipykernel_12451/3795299568.py:95: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  ticket_link_tag = date_row.find('a', text='Tickets')
/var/folders/jt/kpshpf8d1991jhtxl__hj8p00000gn/T/ipykernel_12451/3795299568.py:210: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for p_tag in soup.find_all(['p', 'div', 'span'], text=True):


Extracting all paragraphs...
Extracted 5569 paragraphs and saved to ./output/pittsburgh_events/paragraphs.txt
Closing the WebDriver.


# Downtown Pittsburgh Events

In [22]:

import requests
from bs4 import BeautifulSoup
import os
import urllib.parse
import csv
import json



def extract_events_new_structure(soup, output_path):
    """
    Extracts event details from the new HTML structure and saves them into 'events.csv'.
    
    Args:
        soup (BeautifulSoup): The parsed HTML content.
        output_path (str): The directory where output files will be saved.
    """
    events = []
    
    # Find all divs with class 'eventitem'
    event_items = soup.find_all('div', class_='eventitem')
    
    if not event_items:
        print("No events found in the HTML content.")
        return
    
    for event_div in event_items:
        # Event Name
        event_name_tag = event_div.find('h1')
        event_name = event_name_tag.get_text(strip=True) if event_name_tag else 'N/A'
        
        # Event URL
        event_url_tag = event_name_tag.find('a') if event_name_tag else None
        event_url = urllib.parse.urljoin(url, event_url_tag['href']) if event_url_tag else 'N/A'
        
        # Event Date and Time
        event_date_div = event_div.find('div', class_='eventdate')
        if event_date_div:
            event_date_time = event_date_div.get_text(strip=True)
            # Split date and time if possible
            if '|' in event_date_time:
                event_date, event_time = map(str.strip, event_date_time.split('|'))
            else:
                event_date = event_date_time.strip()
                event_time = 'N/A'
        else:
            event_date = 'N/A'
            event_time = 'N/A'
        
        # Event Description
        description = event_div.get_text(strip=True)
        if event_date_div:
            # Remove date and time from description
            description = description.split(event_date_div.get_text(strip=True))[-1].strip()
        # Remove 'READ MORE' if present
        description = description.replace('READ MORE', '').strip()
        
        # Categories
        category_div = event_div.find('div', class_='category')
        categories = [term.get_text(strip=True) for term in category_div.find_all('div', class_='term')] if category_div else []
        categories_str = ', '.join(categories)
        
        # Location (from JSON-LD script)
        script_tag = event_div.find_next_sibling('script', type='application/ld+json')
        if script_tag:
            json_data = json.loads(script_tag.string)
            location_address = json_data.get('location', {}).get('address', 'N/A')
        else:
            location_address = 'N/A'
        
        # Store the extracted data
        event = {
            'Event Name': event_name,
            'Event URL': event_url,
            'Date': event_date,
            'Time': event_time,
            'Location': location_address,
            'Categories': categories_str,
            'Description': description
        }
        events.append(event)
    
    # Save events to a CSV file
    if events:
        events_file = os.path.join(output_path, 'events.csv')
        keys = ['Event Name', 'Event URL', 'Date', 'Time', 'Location', 'Categories', 'Description']
        with open(events_file, 'w', newline='', encoding='utf-8') as f:
            dict_writer = csv.DictWriter(f, fieldnames=keys)
            dict_writer.writeheader()
            dict_writer.writerows(events)
        print(f"Extracted {len(events)} events and saved to {events_file}")
    else:
        print("No events were extracted.")


def scrape_website(url, output_path):
    # Ensure the output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract all paragraphs and save to paragraphs.txt
        paragraphs = soup.find_all('p')
        paragraphs_text = [para.get_text(strip=True) for para in paragraphs if para.get_text(strip=True)]
        paragraphs_file = os.path.join(output_path, 'paragraphs.txt')
        with open(paragraphs_file, 'w', encoding='utf-8') as f:
            for para in paragraphs_text:
                f.write(para + '\n\n')
        # print(f"Paragraphs saved to {paragraphs_file}")
        
        # Extract all links and save to url.txt
        links = []
        for link in soup.find_all('a', href=True):
            href = link['href']
            full_url = urllib.parse.urljoin(url, href)
            links.append(full_url)
        links_file = os.path.join(output_path, 'url.txt')
        with open(links_file, 'w', encoding='utf-8') as f:
            for link in links:
                f.write(link + '\n')
        # print(f"Links saved to {links_file}")
        
        # Find all PDF and DOCX links and download them
        file_extensions = ('.pdf', '.docx')
        file_links = [link for link in links if link.lower().endswith(file_extensions)]
        if (len(file_links) > 0):
            # Ensure the downloaded_files directory exists

            downloads_folder = os.path.join(output_path, 'downloaded_files')
            if not os.path.exists(downloads_folder):
                os.makedirs(downloads_folder)
        for file_link in file_links:
            file_url = file_link
            file_name = os.path.basename(urllib.parse.urlsplit(file_url).path)
            file_path = os.path.join(downloads_folder, file_name)
            # print(f"Downloading {file_name}...")
            file_response = requests.get(file_url)
            file_response.raise_for_status()
            with open(file_path, 'wb') as f:
                f.write(file_response.content)
            # print(f"Saved {file_name} to {downloads_folder}")
        # print("All PDFs and DOCX files have been downloaded.")
        
        # Extract event details and save to events.csv
        extract_events_new_structure(soup, output_path)
        
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")




url = 'https://downtownpittsburgh.com/events/'
url = 'https://downtownpittsburgh.com/events/?n=10&y=2024&cat=0'
url = 'https://downtownpittsburgh.com/events/?n=11&y=2024&cat=0'
url = 'https://downtownpittsburgh.com/events/?n=12&y=2024&cat=0'
url = 'https://downtownpittsburgh.com/events/?n=1&y=2025&cat=0'
url = 'https://downtownpittsburgh.com/events/?n=2&y=2025&cat=0'
url = 'https://downtownpittsburgh.com/events/?n=3&y=2025&cat=0'
url = 'https://downtownpittsburgh.com/events/?n=4&y=2025&cat=0'
url = 'https://downtownpittsburgh.com/events/?n=5&y=2025&cat=0'
url = 'https://downtownpittsburgh.com/events/?n=6&y=2025&cat=0'
url = 'https://downtownpittsburgh.com/events/?n=7&y=2025&cat=0'
url = 'https://downtownpittsburgh.com/events/?n=8&y=2025&cat=0'
url = 'https://downtownpittsburgh.com/events/?n=9&y=2025&cat=0'
url = 'https://downtownpittsburgh.com/events/?n=10&y=2025&cat=0'

scrape_website(url, './output/downtown_events_feb/')

# Read urls in a text file and scrape each one
with open('./output/downtown_events_oct/url.txt', 'r') as file:
    urls = file.readlines()
    for url in tqdm(urls):
        url = url.strip()
        title = url.split('//')[-1].replace('/', '_')
        scrape_website(url, f'./output/downtown_events_oct/children/{title}')


No events found in the HTML content.


# CMU events

In [ ]:

import requests
from bs4 import BeautifulSoup
import os
import urllib.parse
import csv
import json

def fetch_cmu_events(start_date, timezone="America/New_York", csv_filename="cmu_events.csv"):
    """
    Fetch events from the CMU Events Calendar and store them in a CSV file.
    
    Parameters:
        start_date (str): The start date in YYYYMMDD format (e.g., '20241101').
        timezone (str): The timezone for the events (default is 'America/New_York').
        csv_filename (str): The filename to store the CSV data.
    """
    # Base URL for fetching events
    base_url = "https://events.cmu.edu/live/calendar/view/month/date/"
    
    # URL-encode the timezone to make it safe for the request
    encoded_timezone = timezone.replace("/", "%2F")
    
    # Define the request URL
    url = (f"{base_url}{start_date}?user_tz={encoded_timezone}&template_vars=id,time,title_link,summary,until,repeats,"
           "is_multi_day,is_first_multi_day,multi_day_span,href,tag_classes,category_classes&syntax="
           "%3Cwidget%20type%3D%22events_calendar%22%3E%3Carg%20id%3D%22mini_cal_heat_map%22%3Etrue%3C%2Farg%3E"
           "%3Carg%20id%3D%22thumb_width%22%3E200%3C%2Farg%3E%3Carg%20id%3D%22thumb_height%22%3E200%3C%2Farg%3E"
           "%3Carg%20id%3D%22hide_repeats%22%3Efalse%3C%2Farg%3E%3Carg%20id%3D%22show_groups%22%3Etrue%3C%2Farg%3E"
           "%3Carg%20id%3D%22show_locations%22%3Efalse%3C%2Farg%3E%3Carg%20id%3D%22show_tags%22%3Etrue%3C%2Farg%3E"
           "%3Carg%20id%3D%22month_view_day_limit%22%3E2%3C%2Farg%3E")
    
    # Optional: Add headers to simulate a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }
    
    try:
        # Make the GET request to fetch the events
        response = requests.get(url, headers=headers)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            print(f"response.json(): {response.json()}")
            event_data = response.json().get('days', [])
            print(f'{event_data}')

            # Check if we have any events to write
            if event_data:
                # Open CSV file for writing
                with open(csv_filename, mode='w', newline='') as csv_file:
                    writer = csv.writer(csv_file)
                    
                    # Write header row
                    writer.writerow(["Event ID", "Title", "Start Time", "End Time", "Location", "Summary", "Is Online", "Event URL"])
                    # Loop over the "days" list and extract events
                    for day in event_data:
                        events = day.get('events', [])
                        for event in events:
                            # Write each event's details to the CSV file
                            writer.writerow([
                                event.get("id", ""),
                                event.get("title", ""),
                                event.get("ts_start", ""),
                                event.get("ts_end", ""),
                                event.get("location", ""),
                                event.get("summary", ""),
                                event.get("is_online", False),
                                event.get("href", "")
                            ])
                
                return f"Events successfully saved to {csv_filename}"
            else:
                return "No events found for the specified date."
        else:
            return f"Failed to fetch events, status code: {response.status_code}"
    
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def scrape_website(url, output_path):
    # Ensure the output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract all paragraphs and save to paragraphs.txt
        paragraphs = soup.find_all('p')
        paragraphs_text = [para.get_text(strip=True) for para in paragraphs if para.get_text(strip=True)]
        paragraphs_file = os.path.join(output_path, 'paragraphs.txt')
        with open(paragraphs_file, 'w', encoding='utf-8') as f:
            for para in paragraphs_text:
                f.write(para + '\n\n')
        # print(f"Paragraphs saved to {paragraphs_file}")
        
        # Extract all links and save to url.txt
        links = []
        for link in soup.find_all('a', href=True):
            href = link['href']
            full_url = urllib.parse.urljoin(url, href)
            links.append(full_url)
        links_file = os.path.join(output_path, 'url.txt')
        with open(links_file, 'w', encoding='utf-8') as f:
            for link in links:
                f.write(link + '\n')
        # print(f"Links saved to {links_file}")
        
        # Find all PDF and DOCX links and download them
        file_extensions = ('.pdf', '.docx')
        file_links = [link for link in links if link.lower().endswith(file_extensions)]
        if (len(file_links) > 0):
            # Ensure the downloaded_files directory exists

            downloads_folder = os.path.join(output_path, 'downloaded_files')
            if not os.path.exists(downloads_folder):
                os.makedirs(downloads_folder)
        for file_link in file_links:
            file_url = file_link
            file_name = os.path.basename(urllib.parse.urlsplit(file_url).path)
            file_path = os.path.join(downloads_folder, file_name)
            # print(f"Downloading {file_name}...")
            file_response = requests.get(file_url)
            file_response.raise_for_status()
            with open(file_path, 'wb') as f:
                f.write(file_response.content)
            # print(f"Saved {file_name} to {downloads_folder}")
        # print("All PDFs and DOCX files have been downloaded.")
        
        # Extract event details and save to events.csv
        fetch_cmu_events('20241101', timezone="America/New_York", csv_filename=os.path.join(output_path, 'cmu_events.csv'))

        
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")


url = 'https://events.cmu.edu/month/date/20241101'
scrape_website(url, './output/cmu_events_oct/')

# Read urls in a text file and scrape each one
with open('./output/downtown_events_oct/url.txt', 'r') as file:
    urls = file.readlines()
    for url in tqdm(urls):
        url = url.strip()
        title = url.split('//')[-1].replace('/', '_')
        scrape_website(url, f'./output/downtown_events_oct/children/{title}')


In [4]:
import json
import csv
from datetime import datetime, timedelta

def extract_cmu_events(json_input_path, csv_output_path, url_output_path):
    """
    Extract events from a CMU events JSON file, convert times and locations, and save them into a CSV file.
    Also append event URLs to a URL text file.

    :param json_input_path: Path to the JSON file containing the events data.
    :param csv_output_path: Path to the CSV file where the extracted events will be saved.
    :param url_output_path: Path to the text file where the event URLs will be appended.
    """
    
    # Function to convert Unix timestamp to a readable format in PDT (UTC-7 or UTC-8 depending on DST)
    def convert_timestamp(ts, tz_offset):
        utc_time = datetime.utcfromtimestamp(ts)
        local_time = utc_time + timedelta(seconds=tz_offset)
        return local_time.strftime('%Y-%m-%d %H:%M:%S')

    # Load the JSON data from the input file
    with open(json_input_path, 'r') as json_file:
        data = json.load(json_file)

    # Create a CSV file to store event information
    with open(csv_output_path, 'w', newline='') as csv_file, open(url_output_path, 'a') as url_file:
        writer = csv.writer(csv_file)
        
        # Write the header row
        writer.writerow(["Event ID", "Title", "Start Time", "End Time", "Time Zone", "Summary", "Categories", "Audience", "Event URL"])
        
        # Loop through each day's events
        for day in data.get('days', []):
            for event in day.get('events', []):
                # Extract event information
                event_id = event.get('id', '')
                title = event.get('title', '')
                
                # Convert Unix timestamps to human-readable time in PDT
                start_time = convert_timestamp(event.get('ts_start', 0), event.get('tz_offset', 0))
                end_time = convert_timestamp(event.get('ts_end', 0), event.get('tz_offset', 0))
                
                timezone = event.get('tz_abbrv', 'PDT')  # Default to PDT if not available
                summary = event.get('summary', '')
                categories = ", ".join(event.get('categories', []))
                audience = ", ".join(event.get('categories_audience', []))
                event_url = f"https://events.cmu.edu/{event.get('href', '')}"

                # Write the event information into the CSV file
                writer.writerow([event_id, title, start_time, end_time, timezone, summary, categories, audience, event_url])

                # Append the event URL to the URL text file
                url_file.write(f"{event_url}\n")

    print(f"Events have been successfully saved to {csv_output_path} and URLs appended to {url_output_path}")



# Example usage:
json_input_path = './output/cmu_events/cmu_events_oct.json'
csv_output_path = './output/cmu_events/cmu_events_oct.csv'
url_output_path = './output/cmu_events/events_url.txt'


json_input_path = './output/cmu_events/cmu_events_nov.json'  # Replace with your JSON file path
csv_output_path = './output/cmu_events/cmu_events_nov.csv'       # Replace with your desired CSV output file path
url_output_path = './output/cmu_events/events_url.txt'              # Replace with your desired URL text file path


json_input_path = './output/cmu_events/cmu_events_dec.json'
csv_output_path = './output/cmu_events/cmu_events_dec.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_jan.json'
csv_output_path = './output/cmu_events/cmu_events_jan.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_feb.json'
csv_output_path = './output/cmu_events/cmu_events_feb.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_mar.json'
csv_output_path = './output/cmu_events/cmu_events_mar.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_apr.json'
csv_output_path = './output/cmu_events/cmu_events_apr.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_may.json'
csv_output_path = './output/cmu_events/cmu_events_may.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_jun.json'
csv_output_path = './output/cmu_events/cmu_events_jun.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_jul.json'
csv_output_path = './output/cmu_events/cmu_events_jul.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_aug.json'
csv_output_path = './output/cmu_events/cmu_events_aug.csv'
url_output_path = './output/cmu_events/events_url.txt'
# Call the function
extract_cmu_events(json_input_path, csv_output_path, url_output_path)


Events have been successfully saved to ./output/cmu_events/cmu_events_aug.csv and URLs appended to ./output/cmu_events/events_url.txt


In [14]:
import json
import csv
from datetime import datetime, timedelta

def extract_cmu_events(json_input_path, csv_output_path, url_output_path):
    """
    Extract events from a CMU events JSON file, convert times and locations, and save them into a CSV file.
    Also append event URLs to a URL text file.

    :param json_input_path: Path to the JSON file containing the events data.
    :param csv_output_path: Path to the CSV file where the extracted events will be saved.
    :param url_output_path: Path to the text file where the event URLs will be appended.
    """
    
    # Function to convert Unix timestamp to a readable format in PDT (UTC-7 or UTC-8 depending on DST)
    def convert_timestamp(ts, tz_offset):
        if ts > 0:  # Ensure timestamp is valid
            utc_time = datetime.utcfromtimestamp(ts)
            local_time = utc_time + timedelta(seconds=tz_offset)
            return local_time.strftime('%Y-%m-%d %H:%M:%S')
        else:
            return 'N/A'  # Return 'N/A' if timestamp is invalid

    # Load the JSON data from the input file
    with open(json_input_path, 'r') as json_file:
        data = json.load(json_file)

    # Create a CSV file to store event information
    with open(csv_output_path, 'w', newline='') as csv_file, open(url_output_path, 'a') as url_file:
        writer = csv.writer(csv_file)
        
        # Write the header row
        writer.writerow(["Event ID", "Title", "Start Time", "End Time", "Time Zone", "Summary", "Categories", "Audience", "Event URL"])
        
        # Loop through each day's events
        for day in data.get('days', []):
            for event in day.get('events', []):
                # Extract event information
                event_id = event.get('id', '')
                title = event.get('title', '')
                
                # Convert Unix timestamps to human-readable time in PDT
                start_time = convert_timestamp(event.get('ts_start', 0), event.get('tz_offset', 0))
                end_time = convert_timestamp(event.get('ts_end', 0), event.get('tz_offset', 0))
                
                timezone = event.get('tz_abbrv', 'EDT')  # Default to PDT if not available
                summary = event.get('summary', '')
                categories = ", ".join(event.get('categories', []))
                audience = ", ".join(event.get('categories_audience', []))
                event_url = f"https://events.cmu.edu/{event.get('href', '')}"

                # Write the event information into the CSV file
                writer.writerow([event_id, title, start_time, end_time, timezone, summary, categories, audience, event_url])

                # Append the event URL to the URL text file
                url_file.write(f"{event_url}\n")

    print(f"Events have been successfully saved to {csv_output_path} and URLs appended to {url_output_path}")


# # Example usage:
json_input_path = './output/cmu_events/cmu_events_oct.json'
csv_output_path = './output/cmu_events/cmu_events_oct.csv'
url_output_path = './output/cmu_events/events_url.txt'


json_input_path = './output/cmu_events/cmu_events_nov.json'  # Replace with your JSON file path
csv_output_path = './output/cmu_events/cmu_events_nov.csv'       # Replace with your desired CSV output file path
url_output_path = './output/cmu_events/events_url.txt'              # Replace with your desired URL text file path

json_input_path = './output/cmu_events/cmu_events_dec.json'
csv_output_path = './output/cmu_events/cmu_events_dec.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_jan.json'
csv_output_path = './output/cmu_events/cmu_events_jan.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_feb.json'
csv_output_path = './output/cmu_events/cmu_events_feb.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_mar.json'
csv_output_path = './output/cmu_events/cmu_events_mar.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_apr.json'
csv_output_path = './output/cmu_events/cmu_events_apr.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_may.json'
csv_output_path = './output/cmu_events/cmu_events_may.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_jun.json'
csv_output_path = './output/cmu_events/cmu_events_jun.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_jul.json'
csv_output_path = './output/cmu_events/cmu_events_jul.csv'
url_output_path = './output/cmu_events/events_url.txt'

json_input_path = './output/cmu_events/cmu_events_aug.json'
csv_output_path = './output/cmu_events/cmu_events_aug.csv'
url_output_path = './output/cmu_events/events_url.txt'
# Call the function
extract_cmu_events(json_input_path, csv_output_path, url_output_path)

Events have been successfully saved to ./output/cmu_events/cmu_events_oct.csv and URLs appended to ./output/cmu_events/events_url.txt


In [65]:
import os
import requests
from bs4 import BeautifulSoup
import urllib.parse
import json
from tqdm import tqdm

def scrape_event_page(url, output_path):
    # Ensure the output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract the meta information (for description, title, etc.)
        title = soup.find('meta', property='og:title')
        title_text = title['content'] if title else 'No title found'

        description = soup.find('meta', property='og:description')
        description_text = description['content'] if description else 'No description found'

        event_url = soup.find('meta', property='og:url')
        event_url_text = event_url['content'] if event_url else 'No URL found'

        image_url = soup.find('meta', property='og:image')
        image_url_text = image_url['content'] if image_url else 'No image URL found'
        
        # Extract JSON-LD data for structured event info (if available)
        json_ld_script = soup.find('script', type='application/ld+json')
        if json_ld_script:
            try:
                event_data = json.loads(json_ld_script.string)
                event_start = event_data.get('startDate', 'No start date found')
                event_end = event_data.get('endDate', 'No end date found')
            except json.JSONDecodeError:
                event_data = {}
                event_start = 'No start date found'
                event_end = 'No end date found'
        else:
            event_data = {}
            event_start = 'No start date found'
            event_end = 'No end date found'

        # Save extracted data to a text file
        event_details_file = os.path.join(output_path, 'event_details.txt')
        with open(event_details_file, 'w', encoding='utf-8') as f:
            f.write(f"Title: {title_text}\n")
            f.write(f"Description: {description_text}\n")
            f.write(f"Event URL: {event_url_text}\n")
            f.write(f"Image URL: {image_url_text}\n")
            f.write(f"Start Date: {event_start}\n")
            f.write(f"End Date: {event_end}\n")
        print(f"Event details saved to {event_details_file}")

        # Extract all links and save to url.txt
        links = []
        for link in soup.find_all('a', href=True):
            href = link['href']
            full_url = urllib.parse.urljoin(url, href)
            links.append(full_url)
        links_file = os.path.join(output_path, 'url.txt')
        with open(links_file, 'w', encoding='utf-8') as f:
            for link in links:
                f.write(link + '\n')
        print(f"Links saved to {links_file}")
        
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")


with open('./output/cmu_events/events_url.txt', 'r') as file:
    urls = file.readlines()
    for url in tqdm(urls):
        url = url.strip()
        title = url.split('//')[-1].replace('/', '_')
        scrape_event_page(url, f'./output/cmu_events/children/{title}')

  0%|          | 1/585 [00:00<03:12,  3.04it/s]

Event details saved to ./output/cmu_events/children/events.cmu.edu_event_11977-womens-volleyball-case-western-reserve-vs-carnegie/event_details.txt
Links saved to ./output/cmu_events/children/events.cmu.edu_event_11977-womens-volleyball-case-western-reserve-vs-carnegie/url.txt


  1%|          | 3/585 [00:00<02:18,  4.20it/s]

Event details saved to ./output/cmu_events/children/events.cmu.edu_event_12531-halloween-trick-or-treat-with-tartan-football/event_details.txt
Links saved to ./output/cmu_events/children/events.cmu.edu_event_12531-halloween-trick-or-treat-with-tartan-football/url.txt
Event details saved to ./output/cmu_events/children/events.cmu.edu_event_9777-provosts-office-hours/event_details.txt
Links saved to ./output/cmu_events/children/events.cmu.edu_event_9777-provosts-office-hours/url.txt


  1%|          | 5/585 [00:01<02:12,  4.37it/s]

...

An error occurred: 403 Client Error: Forbidden for url: https://events.cmu.edu/event/12337-summer-all-two-mini-6-faculty-course
An error occurred: 403 Client Error: Forbidden for url: https://events.cmu.edu/event/12338-summer-all-two-mini-6-final-exams
An error occurred: 403 Client Error: Forbidden for url: https://events.cmu.edu/event/12341-summer-all-two-mini-6-faculty-course


100%|█████████▉| 583/585 [02:15<00:00, 10.73it/s]

An error occurred: 403 Client Error: Forbidden for url: https://events.cmu.edu/event/12339-summer-all-two-mini-6-last-day-of-classes
An error occurred: 403 Client Error: Forbidden for url: https://events.cmu.edu/event/12340-summer-all-two-mini-6-voucher-deadline
An error occurred: 403 Client Error: Forbidden for url: https://events.cmu.edu/event/12337-summer-all-two-mini-6-faculty-course


100%|██████████| 585/585 [02:15<00:00,  4.32it/s]

An error occurred: 403 Client Error: Forbidden for url: https://events.cmu.edu/event/12338-summer-all-two-mini-6-final-exams
An error occurred: 403 Client Error: Forbidden for url: https://events.cmu.edu/event/12336-summer-all-two-mini-6-final-grades-due-by-4pm


In [81]:
def extract_events_pghcitypaper(soup, output_path, base_url):
    """
    Extracts event details from the pghcitypaper.com website and saves them into 'events.csv'.

    Args:
        soup (BeautifulSoup): The parsed HTML content.
        output_path (str): The directory where output files will be saved.
        base_url (str): The base URL of the website.
    """
    events = []

    # Find all <li> elements with class 'fdn-pres-item'
    event_items = soup.find_all('li', class_='fdn-pres-item')

    if not event_items:
        print("No events found in the HTML content.")
        return

    for event_item in event_items:
        # Event Name and URL
        headline_p = event_item.find('p', class_='fdn-teaser-headline')
        if headline_p:
            event_name_tag = headline_p.find('a')
            event_name = event_name_tag.get_text(strip=True) if event_name_tag else 'N/A'
            event_url = urllib.parse.urljoin(base_url, event_name_tag['href']) if event_name_tag else 'N/A'
        else:
            event_name = 'N/A'
            event_url = 'N/A'

        # Event Date and Time
        date_time_p = event_item.find('p', class_='fdn-teaser-subheadline')
        event_date_time = date_time_p.get_text(strip=True) if date_time_p else 'N/A'

        # Venue Name
        venue_p = event_item.find('p', class_='fdn-event-teaser-location')
        if venue_p:
            venue_a = venue_p.find('a')
            venue_name = venue_a.get_text(strip=True) if venue_a else 'N/A'
        else:
            venue_name = 'N/A'

        # Venue Address
        address_p = event_item.find('p', class_='fdn-inline-split-list fdn-teaser-infoline')
        if address_p:
            address_spans = address_p.find_all('span')
            if len(address_spans) >= 1:
                venue_address = address_spans[0].get_text(strip=True)
            else:
                venue_address = 'N/A'
        else:
            venue_address = 'N/A'

        # Price
        price_div = event_item.find('div', class_='fdn-pres-details-split fdn-teaser-infoline')
        if price_div:
            price_span = price_div.find('span')
            price = price_span.get_text(strip=True) if price_span else 'N/A'
        else:
            price = 'N/A'

        # Category
        category_p = event_item.find('p', class_='fdn-teaser-infoline uk-text-break uk-margin-xsmall-top')
        if category_p:
            category_a = category_p.find('a')
            category = category_a.get_text(strip=True) if category_a else 'N/A'
        else:
            category = 'N/A'

        # Description
        description_div = event_item.find('div', class_='fdn-teaser-description')
        description = description_div.get_text(strip=True) if description_div else 'N/A'

        # Store the extracted data
        event = {
            'Event Name': event_name,
            'Event URL': event_url,
            'Date and Time': event_date_time,
            'Venue Name': venue_name,
            'Venue Address': venue_address,
            'Price': price,
            'Category': category,
            'Description': description
        }
        events.append(event)

    # Save events to a CSV file
    if events:
        events_file = os.path.join(output_path, 'events.csv')
        keys = ['Event Name', 'Event URL', 'Date and Time', 'Venue Name', 'Venue Address', 'Price', 'Category', 'Description']
        with open(events_file, 'w', newline='', encoding='utf-8') as f:
            dict_writer = csv.DictWriter(f, fieldnames=keys)
            dict_writer.writeheader()
            dict_writer.writerows(events)
        print(f"Extracted {len(events)} events and saved to {events_file}")
    else:
        print("No events were extracted.")


def scrape_website(url, output_path):
    # Ensure the output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract all paragraphs and save to paragraphs.txt
        paragraphs = soup.find_all('p')
        paragraphs_text = [para.get_text(strip=True) for para in paragraphs if para.get_text(strip=True)]
        paragraphs_file = os.path.join(output_path, 'paragraphs.txt')
        with open(paragraphs_file, 'w', encoding='utf-8') as f:
            for para in paragraphs_text:
                f.write(para + '\n\n')
        print(f"Paragraphs saved to {paragraphs_file}")
        
        # Extract all links and save to url.txt
        links = []
        for link in soup.find_all('a', href=True):
            href = link['href']
            full_url = urllib.parse.urljoin(url, href)
            links.append(full_url)
        links_file = os.path.join(output_path, 'url.txt')
        with open(links_file, 'w', encoding='utf-8') as f:
            for link in links:
                f.write(link + '\n')
        print(f"Links saved to {links_file}")
        
        # Find all PDF and DOCX links and download them
        file_extensions = ('.pdf', '.docx')
        file_links = [link for link in links if link.lower().endswith(file_extensions)]
        print(file_links)
        if (len(file_links) > 0):
            downloads_folder = os.path.join(output_path, 'downloaded_files')
            if not os.path.exists(downloads_folder):
                os.makedirs(downloads_folder)
        for file_link in file_links:
            file_url = file_link
            file_name = os.path.basename(urllib.parse.urlsplit(file_url).path)
            file_path = os.path.join(downloads_folder, file_name)
            print(f"Downloading {file_name}...")
            file_response = requests.get(file_url)
            file_response.raise_for_status()
            with open(file_path, 'wb') as f:
                f.write(file_response.content)
            print(f"Saved {file_name} to {downloads_folder}")
        print("All PDFs and DOCX files have been downloaded.")
        
        # Extract event details and save to events.csv
        extract_events_pghcitypaper(soup, output_path, url)
        
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")


url = 'https://www.pghcitypaper.com/pittsburgh/EventSearch?sortType=date&v=d'

# scrape_website(url, './output/pghcitypaper_events/')

with open('./output/pghcitypaper_events/url.txt', 'r') as file:
    urls = file.readlines()
    for url in tqdm(urls):
        url = url.strip()
        title = url.split('//')[-1].replace('/', '_')
        scrape_website(url, f'./output/pghcitypaper_events/children/{title}')

  0%|          | 1/356 [00:00<03:18,  1.79it/s]

Paragraphs saved to ./output/pghcitypaper_events/children/twitter.com_PGHCityPaper/paragraphs.txt
Links saved to ./output/pghcitypaper_events/children/twitter.com_PGHCityPaper/url.txt
[]
All PDFs and DOCX files have been downloaded.
No events found in the HTML content.


  1%|          | 2/356 [00:00<02:28,  2.38it/s]

Paragraphs saved to ./output/pghcitypaper_events/children/www.facebook.com_PittsburghCityPaper?sk=wall/paragraphs.txt
Links saved to ./output/pghcitypaper_events/children/www.facebook.com_PittsburghCityPaper?sk=wall/url.txt
[]
All PDFs and DOCX files have been downloaded.
No events found in the HTML content.


  1%|          | 3/356 [00:01<04:01,  1.46it/s]

...

 54%|█████▍    | 194/356 [00:50<00:27,  5.97it/s]

Paragraphs saved to ./output/pghcitypaper_events/children/www.pghcitypaper.com_pittsburgh_halloween-season-of-the-sketch_Event?oid=26694744/paragraphs.txt
Links saved to ./output/pghcitypaper_events/children/www.pghcitypaper.com_pittsburgh_halloween-season-of-the-sketch_Event?oid=26694744/url.txt
[]
All PDFs and DOCX files have been downloaded.
Extracted 1 events and saved to ./output/pghcitypaper_events/children/www.pghcitypaper.com_pittsburgh_halloween-season-of-the-sketch_Event?oid=26694744/events.csv
Paragraphs saved to ./output/pghcitypaper_events/children/www.pghcitypaper.com_pittsburgh_halloween-season-of-the-sketch_Event?oid=26694744/paragraphs.txt
Links saved to ./output/pghcitypaper_events/children/www.pghcitypaper.com_pittsburgh_halloween-season-of-the-sketch_Event?oid=26694744/url.txt
[]
All PDFs and DOCX files have been downloaded.
Extracted 1 events and saved to ./output/pghcitypaper_events/children/www.pghcitypaper.com_pittsburgh_halloween-season-of-the-sketch_Event?oid=

 55%|█████▍    | 195/356 [00:50<00:41,  3.86it/s]

Paragraphs saved to ./output/pghcitypaper_events/children/www.pghcitypaper.com_pittsburgh_arcade-comedy-theater_Location?oid=1623712/paragraphs.txt
Links saved to ./output/pghcitypaper_events/children/www.pghcitypaper.com_pittsburgh_arcade-comedy-theater_Location?oid=1623712/url.txt
[]
All PDFs and DOCX files have been downloaded.
Extracted 8 events and saved to ./output/pghcitypaper_events/children/www.pghcitypaper.com_pittsburgh_arcade-comedy-theater_Location?oid=1623712/events.csv


OSError: [Errno 63] File name too long: './output/pghcitypaper_events/children/l.facebook.com_l.php?u=https%3A%2F%2Fwww.showclix.com%2Fevent%2Fhalloween-season-of-the-sketch-2410%3Ffbclid%3DIwZXh0bgNhZW0CMTAAAR08zTRgoD2BoCyWrx-bHHCbGDPkE5L-Q44u8ajEsMsVmM-DjqJu7k0S7wM_aem_P0XoKR4-Y4VOovQlTPGNOg&h=AT0qIpn-UffVImGex1UFYPWWs2qFwEw77bfYyjGweMghg20C8bfodosfOq2vOUIwaOG91Bsqd941eJFezu19TlrmRq0Njip-FDR0M11FMvalTYfqxTX0934NYJN4c06q&__tn__=-UK-R&c[0]=AT33nWt55T_SMwTGmkUa8GDkGHT3KWtouGrDQuUvSFrwLv3xILeqx8Sb0Y8f0AOnBKQJ9ABZTbsYvYc5mdC6JkQbOvDDzkhp9H8aYFDUlmaJCynmfObMXhYBllZjXn7xZBI0Fp09kermbj5Qe3IcOTFYmQgptc860nkZC-lx8kVdF9KMTuAWoUJJndsL-YwnBOohlY28BhH9HRzmV9fBxxpZL1JyY2idkR9-9g'